In [1]:
import pandas as pd
import numpy as np

# ETL

En el siguiente trabajo de ETL cada sección incluye el trabajo de los cuatro dataFrames. Por este motivo, cada vez que hablemos de un procedimiento nos estaremos refiriendo a todos los data frames por igual. Si en el algún caso el proceso entre datasets difiere, lo indicaremos en la correspondiente sección.

## Carga de datos

In [2]:
dfa = pd.read_csv('..\\Datasets\\amazon_prime_titles-score.csv')

In [3]:
#dfa.head()

In [4]:
dfd = pd.read_csv('..\\Datasets\\disney_plus_titles-score.csv')

In [5]:
#dfd.head()

In [6]:
dfd = pd.read_csv('..\\Datasets\\disney_plus_titles-score.csv')

In [7]:
dfh = pd.read_csv('..\\Datasets\\hulu_titles-score (2).csv')

In [8]:
dfn = pd.read_csv('..\\Datasets\\netflix_titles-score.csv')

## Chequeo de valores nulos

In [9]:
dfa.isna().any().any()

True

In [10]:
def get_nan_in_df(dataFrame):
    # get a Pandas series with all null values in the data frame.
    nanValues = dataFrame.isna().sum().sort_values(ascending=False)
    nanValuesPrct = (nanValues/len(dataFrame) )*100
    
    return nanValuesPrct

In [11]:
#get_nan_in_df(dfa)

In [12]:
#get_nan_in_df(dfd)

In [13]:
dfa.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,99
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,37
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,20
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",27
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,75


### Generar campo id

- Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [14]:
#dfa.tail()

**Insertamos una nueva columna en la primera posición del DataFrame, con nombre 'id' y usamos una función anónima para darle el valor a cada celda.** 

In [15]:
dfa.insert(0,'id', dfa.show_id.apply(lambda x: f'a{x}'))

In [16]:
dfd.insert(0,'id', dfd.show_id.apply(lambda x: f'd{x}'))

In [17]:
dfh.insert(0,'id', dfh.show_id.apply(lambda x: f'h{x}'))

In [18]:
dfn.insert(0,'id', dfn.show_id.apply(lambda x: f'n{x}'))

In [19]:
#dfa.head()

### Reemplazar valores nulos en el campo rating por el char "G"

In [20]:
#dfa.rating.value_counts()

**El procedimiento es trivial, simplemente procedimos a tratar los nan con el método fillna de pandas**

In [21]:
dfa.rating = dfa.rating.fillna('G')
dfd.rating = dfd.rating.fillna('G')
dfh.rating = dfh.rating.fillna('G')
dfn.rating = dfn.rating.fillna('G')

In [22]:
dfn.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ns1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29
1,ns2,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",86
2,ns3,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,75
3,ns4,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",49
4,ns5,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,92


### Formatear fechas

**Las fechas deberán tener el formato `AAAA-mm-dd`**

In [23]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            9668 non-null   object
 1   show_id       9668 non-null   object
 2   type          9668 non-null   object
 3   title         9668 non-null   object
 4   director      7586 non-null   object
 5   cast          8435 non-null   object
 6   country       672 non-null    object
 7   date_added    155 non-null    object
 8   release_year  9668 non-null   int64 
 9   rating        9668 non-null   object
 10  duration      9668 non-null   object
 11  listed_in     9668 non-null   object
 12  description   9668 non-null   object
 13  score         9668 non-null   int64 
dtypes: int64(2), object(12)
memory usage: 1.0+ MB


In [24]:
# dfa.date_added.apply(lambda x: type(x) == str)
# dfd.date_added.apply(lambda x: type(x) == str)
# dfh.date_added.apply(lambda x: type(x) == str)
# dfn.date_added.apply(lambda x: type(x) == str)

In [25]:
# dfh.date_added.apply(lambda x: type(x) == str)
# dfn.date_added.apply(lambda x: type(x) == str)

In [26]:
dfa.date_added = pd.to_datetime(dfa.date_added)
dfd.date_added = pd.to_datetime(dfd.date_added)
dfh.date_added = pd.to_datetime(dfh.date_added)
dfn.date_added = pd.to_datetime(dfn.date_added)

In [27]:
dfn.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,ns1,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",29
1,ns2,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",86
2,ns3,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,75
3,ns4,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",49
4,ns5,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,92


## Normalizar los campos de texto

- todos los campos de texto deberán estar en minúsculas, sin excepción.

In [28]:
dfa.dtypes

id                      object
show_id                 object
type                    object
title                   object
director                object
cast                    object
country                 object
date_added      datetime64[ns]
release_year             int64
rating                  object
duration                object
listed_in               object
description             object
score                    int64
dtype: object

Para llevar adelante este paso, nos servimos de la función `all_columns_to_lower_case()` que recibe un data frame. Esta función itera sobre todas las columnas con tipo de dato string. Además, lleva el texto de cada celda a minúsculas, utilizando a la función auxiliar `to_lower_case()`. Finalmente retorna el data frame procesado.

Por su parte `to_lower_case()` discrimina según el tipo de dato recibido como parámetro (proveniente de cada celda seleccionada), ya que varios de los datos en las columnas de texto son nan. Si el dato es de tipo string, lo normaliza y retorna el dato procesado; de lo contrario, devuelve el dato ingresado como parámetro(nan).

In [29]:
def all_columns_to_lower_case(DataFrame):
    for columna in DataFrame.select_dtypes('object').columns:
        DataFrame[columna] = DataFrame[columna].apply(lambda x: to_lower_case(x))
    return DataFrame

In [30]:
def to_lower_case(cellValue):
    if type(cellValue) == str:
        value = cellValue.lower()
        return value
    else:
        return cellValue

In [31]:
dfa = all_columns_to_lower_case(dfa)
dfd = all_columns_to_lower_case(dfd)
dfh = all_columns_to_lower_case(dfh)
dfn = all_columns_to_lower_case(dfn)

In [32]:
#dfn

### Separar los datos de la columna 'duration'

- Separar en dos campos: duration_int y duration_type. El primero será un integer, el segundo un string que indique la unidad de medición de duración: min (minutos) o season (temporadas)



En primer lugar, utilizamos el método `insert()` de Pandas para crear los nuevos campos, decidimos ubicarlos justo a la derecha de la columna original y le pasamos los datos de esta a cada celda de las nuevas columnas.

In [33]:
dfa.insert(loc = 11, column = 'duration_int', value = dfa.duration)
dfa.insert(loc = 12, column = 'duration_type', value = dfa.duration)

In [34]:
dfd.insert(loc = 11, column = 'duration_int', value = dfd.duration)
dfd.insert(loc = 12, column = 'duration_type', value = dfd.duration)

In [35]:
dfh.insert(loc = 11, column = 'duration_int', value = dfh.duration)
dfh.insert(loc = 12, column = 'duration_type', value = dfh.duration)

In [36]:
dfn.insert(loc = 11, column = 'duration_int', value = dfn.duration)
dfn.insert(loc = 12, column = 'duration_type', value = dfn.duration)

In [37]:
dfa.head()

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration_int,duration_type,listed_in,description,score
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,113 min,113 min,"comedy, drama",a small fishing village must procure a local d...,99
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,110 min,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37
2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,74 min,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20
3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,69 min,69 min,documentary,"pink breaks the mold once again, bringing her ...",27
4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,45 min,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,75


In [38]:
dfa.duration.value_counts()

1 season      1342
2 seasons      229
90 min         223
91 min         220
92 min         203
              ... 
21 seasons       1
485 min          1
550 min          1
181 min          1
191 min          1
Name: duration, Length: 219, dtype: int64

En segundo lugar, para los data frames de Amazon y Disney, en el campo duration_int separamos los datos con la función `split` de Python, como se describe abajo. Mientras que para los data frames de Hulu y Nextflix debimos utilizar la función ` get_int_value()` para diferenciar strings de valores nulos y poder trabajar con estas sin complicaciones.

In [39]:
# La función anónima itera sobre cada valor de la columna duration_int, separa la cadena y selecciona la primera slice
# y finalmente castea al tipo de dato solicitado, un int.
dfa.duration_int = dfa.duration_int.apply(lambda x: int(x.split(' ')[0]))
dfd.duration_int = dfd.duration_int.apply(lambda x: int(x.split(' ')[0]))
#dfh.duration_int = dfh.duration_int.apply(lambda x: int(x.split(' ')[0]))
#dfn.duration_int = dfn.duration_int.apply(lambda x: int(x.split(' ')[0]))

In [40]:
# def to_lower_case(cellValue):
#     if type(cellValue) == str:
#         value = cellValue.lower()
#         return value
#     else:
#         return cellValue

In [41]:
def get_int_value(cellValue): 
    if type(cellValue) == str:
        value = cellValue.split(' ')[0]
        return int(value)
    else:
        return np.nan

In [42]:
dfh.duration_int = dfh.duration.apply(lambda x: get_int_value(x))

In [43]:
dfn.duration_int = dfn.duration.apply(lambda x: get_int_value(x))

Otro paso adicional que debimos realizar con los dataframes de Hulu y Netflix fue convertir el tipo de datos a int. Para optimizar el uso la memoria, Pandas coerciona el tipo de dato int a float cuando se intenta volcar valores nan y enteros en un mismo campo y al mismo tiempo. Por esta razón tuvimos que expicitar el tipo de dato que necesitábamos dentro de la serie.

In [44]:
dfh.duration_int = pd.Series(dfh.duration_int, dtype=pd.Int32Dtype())

In [45]:
dfn.duration_int = pd.Series(dfn.duration_int, dtype=pd.Int32Dtype())

En tercer lugar, separamos los datos de la columna duration_type para quedarnos con la substring que contiene el tipo de duración de los shows (min, season). Todo este proceso es análogo al utilizado en la columna duration_int.
Aquí también tuvimos que prodecer de manera diferentes entre los data frames de Amazon y Disney, por un lado y Hulu y Netflix por el otro.

In [46]:
# Esta función es muy similar a la anterior, pero no necesita convertir el tipo de dato y toma la segunda slice de la string 
# original
dfa.duration_type = dfa.duration_type.apply(lambda x: x.split(' ')[1])
dfd.duration_type = dfd.duration_type.apply(lambda x: x.split(' ')[1])
#dfh.duration_type = dfh.duration_type.apply(lambda x: x.split(' ')[1])
#dfn.duration_type = dfn.duration_type.apply(lambda x: x.split(' ')[1])

In [47]:
def get_text_slice_value(cellValue):
    if type(cellValue) == str:
        value = cellValue.split(' ')[1]
        return value
    else:
        return cellValue

In [48]:
dfh.duration_type= dfh.duration_type.apply(lambda x: get_text_slice_value(x))

In [49]:
dfn.duration_type = dfn.duration_type.apply(lambda x: get_text_slice_value(x))

En cuarto y último lugar normalizamos el tipo de duración, al llevar el valor 'seasons' a 'season'.

In [50]:
def replace_plural(cellValue):
    if type(cellValue) == str:
        if cellValue[-1] == 's':
            cellValue =  cellValue[:-1]   
            
    return cellValue

In [51]:
dfa.duration_type = dfa.duration_type.apply(replace_plural)

In [52]:
dfd.duration_type = dfd.duration_type.apply(replace_plural)

In [53]:
dfh.duration_type = dfh.duration_type.apply(replace_plural)

In [54]:
dfn.duration_type = dfn.duration_type.apply(replace_plural)

In [55]:
#dfn.duration_type.value_counts()

In [56]:
#dfa

In [57]:
#dfd

In [58]:
#dfh

In [59]:
#dfn

## Guardar los datos

In [60]:
dfa.to_csv('..\\Amazon.csv', index = False)

In [61]:
dfd.to_csv('..\\Disney.csv', index = False)

In [62]:
dfh.to_csv('..\\Hulu.csv', index = False)

In [63]:
dfn.to_csv('..\\Netflix.csv', index = False)